# EEP 153: Project 2
# Minimum Cost Diet: Vegans vs Vegetarians vs Omnivores
## By Keanna, Claire, Janice, Vincent and Youssef

## Population of Interest

### We are looking at different types of diets for people in the United States and comparing their minimum costs subject to nutritional constraints. The types of diets we will study are: Omnivorous, Vegetarian, and Vegan.

In [1]:
# import necessary packages
!pip install -r requirements.txt
!pip install wbdata

import ndb
import pandas as pd
import warnings
from  scipy.optimize import linprog as lp
import numpy as np
import wbdata 

## Data on Food Prices

In [2]:
# import google spreadsheet with food prices for each of the 3 diets

In [11]:
omni_p = pd.read_csv('Data/omnivore food prices.csv', dtype=str)
vegi_p = pd.read_csv('Data/vegetarian food prices.csv', dtype=str)
veg_p = pd.read_csv('Data/vegan food prices.csv', dtype=str)

In [12]:
print(len(omni_p))
omni_p.head()

85


,Food,Quantity,Units,Price,Date,Location,NDB
0,"Flour, white, all purpose, per lb. (453.6 gm)",1,lb,0.439,Jan 2019,DOL,45181883
1,"Rice, white, long grain, uncooked, per lb. (45...",1,lb,0.73,Jan 2019,DOL,45285910
2,"Spaghetti and macaroni, per lb. (453.6 gm)",1,lb,1.217,Jan 2019,DOL,45213192
3,"Bread, white, pan, per lb. (453.6 gm)",1,lb,1.274,Jan 2019,DOL,18069
4,"Bread, whole wheat, pan, per lb. (453.6 gm)",1,lb,1.94,Jan 2019,DOL,18075


In [13]:
print(len(vegi_p))
vegi_p.tail()

58


,Food,Quantity,Units,Price,Date,Location,NDB
53,Navy Beans,1,lbs,1.49,Oct 2018,Target,45273292
54,Sugar,4,lbs,2.39,Oct 2018,Target,19336
55,Pancake Flour,32,oz,1.99,Oct 2018,Trader Joes,45054364
56,Beets,1,lbs,2.49,Oct 2018,Good Eggs,11080
57,Almond Milk,64,oz,3,Feb 2019,Safeway,45209232


In [14]:
print(len(veg_p))
veg_p.tail()

51


,Food,Quantity,Units,Price,Date,Location,NDB
46,Olive Oil,25.4,Fl. Oz.,9.09,Feb 2019,Signature select,45239582
47,Avocados,4,ct,3.99,Feb 2019,Amazon,45346458
48,Tofu,14,oz,1.99,Feb 2019,Whole Food 365,45267096
49,Nutritional Yeast,10,oz,9.44,Feb 2019,NowFoods,45360902
50,Almond Milk,64,oz,3,Feb 2019,Safeway,45209232


## Data on Nutritional Content

In [15]:
# join food price data with nutritional content data based on US recommendations

In [16]:
user = "casimirfunk"
apikey = {'casimirfunk':"inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"}

In [18]:
D = {}
for food in  omni_p.Food.tolist():
    try:
        NDB = omni_p.loc[omni_p.Food==food,:].NDB
        D[food] = ndb.ndb_report(apikey[user],NDB).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find NDB Code %s for food %s." % (food,NDB))
        

D = pd.DataFrame(D,dtype=float)

D

,"Flour, white, all purpose, per lb. (453.6 gm)","Rice, white, long grain, uncooked, per lb. (453.6 gm)","Spaghetti and macaroni, per lb. (453.6 gm)","Bread, white, pan, per lb. (453.6 gm)","Bread, whole wheat, pan, per lb. (453.6 gm)","Cookies, chocolate chip, per lb. (453.6 gm)","Ground chuck, 100% beef, per lb. (453.6 gm)","Ground beef, 100% beef, per lb. (453.6 gm)","Ground beef, lean and extra lean, per lb. (453.6 gm)","All uncooked ground beef, per lb. (453.6 gm)",...,Peas,Chickpeas,Almonds,Chia,Olive Oil,Avocados,Tofu,Nutritional Yeast,Yoghurt,Almond Milk
Caffeine,NaN,NaN,NaN,0.000,0.000,12.000,NaN,NaN,NaN,NaN,...,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Calcium, Ca",67.000,27.000,0.000,144.000,161.000,43.000,18.000,0.00,0.00,NaN,...,25.000,31.00,269.000,633.00,NaN,0.00,76.00,40.000,189.00,185.00
"Carbohydrate, by difference",73.330,80.000,73.210,49.420,42.710,66.100,0.000,0.00,0.00,0.00,...,14.450,15.38,21.550,41.67,0.00,10.00,2.53,33.330,5.73,0.62
Cholesterol,0.000,0.000,0.000,0.000,0.000,0.000,71.000,62.00,54.00,71.00,...,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.000,22.00,0.00
Energy,367.000,356.000,357.000,266.000,252.000,497.000,250.000,205.00,125.00,250.00,...,81.000,92.00,579.000,500.00,800.00,167.00,101.00,400.000,106.00,12.00
"Fatty acids, total monounsaturated",NaN,0.000,0.000,0.599,0.620,13.077,NaN,NaN,NaN,NaN,...,0.035,NaN,31.551,0.00,66.67,NaN,NaN,NaN,NaN,NaN
"Fatty acids, total polyunsaturated",NaN,0.000,0.890,1.602,1.592,2.620,NaN,NaN,NaN,NaN,...,0.187,NaN,12.329,25.00,6.67,NaN,NaN,NaN,NaN,NaN
"Fatty acids, total saturated",0.000,0.000,0.000,0.698,0.722,8.316,8.040,6.25,1.79,8.04,...,0.071,0.00,3.802,0.00,13.33,1.67,0.63,0.000,3.96,0.00
"Fatty acids, total trans",0.000,0.000,0.000,0.027,0.020,NaN,NaN,0.00,0.00,NaN,...,0.000,0.00,0.015,0.00,0.00,0.00,0.00,0.000,0.00,0.00
"Fiber, total dietary",3.300,2.200,3.600,2.700,6.000,NaN,NaN,0.00,0.00,NaN,...,5.700,3.10,12.500,33.30,NaN,6.70,1.30,20.000,0.00,0.30


In [19]:
# Convert price and quantity values into floats from strings
for df in [omni_p, vegi_p, veg_p]:
    df['Price'] = df['Price'].astype(float)
    df['Quantity'] = df['Quantity'].astype(float)

In [20]:
# Convert food quantities to NDB units
veg_p['NDB Quantity'] = veg_p[['Quantity','Units']].T.apply(lambda x : ndb.ndb_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
veg_p['NDB Price'] = veg_p['Price']/veg_p['NDB Quantity']

veg_p.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
veg_prices = veg_p.groupby('Food')['NDB Price'].min()

veg_prices.head()

Food
Almond Milk                    0.16534669663865817 / hectogram
Almonds                         2.2094327075691726 / hectogram
Apples Red Delicious Large      0.5048585804033696 / hectogram
Asparagus Green                 0.8796444261176615 / hectogram
Avocados                      0.009975000000000001 / hectogram
Name: NDB Price, dtype: object

In [38]:
# Choose sex/age group:
group = "F 19-30"

# Define *minimums*
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

# Define *maximums*
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]


## Minimum Cost Solutions

In [39]:
# solve for minimum cost diets for omnivores

# Convert food quantities to NDB units
omni_p['NDB Quantity'] = omni_p[['Quantity','Units']].T.apply(lambda x : ndb.ndb_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
omni_p['NDB Price'] = omni_p['Price']/omni_p['NDB Quantity']

omni_p.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
omni_prices = omni_p.groupby('Food')['NDB Price'].min()

In [40]:
# solve for minimum cost diets for omnivores

tol = 1e-2 # Numbers in solution smaller than this (in absolute value) treated as zeros

c = omni_prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([-Amin,Amax])

b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

# Now solve problem!
result = lp(c, A, b, method='interior-point')

# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))
print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())

Cost of diet for F 19-30 is $2.26 per day.

You'll be eating (in 100s of grams or milliliters):
Food
Avocados                                                 3.853701
Rice, white, long grain, uncooked, per lb. (453.6 gm)    0.352921
Liver (Pork)                                             0.823036
Bananas, per lb. (453.6 gm)                              2.330894
Milk (Whole)                                             2.082329
Almonds                                                  0.543212
Cabbage                                                  1.122871
Beans, dried, any type, all sizes, per lb. (453.6 gm)    0.480319
dtype: float64

With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000000          2000.0
Protein                           65.859456            46.0
Fiber, total dietary              65.725404            28.0
Fo

In [41]:
# solve for vegetarians

# Convert food quantities to NDB units
vegi_p['NDB Quantity'] = vegi_p[['Quantity','Units']].T.apply(lambda x : ndb.ndb_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
vegi_p['NDB Price'] = vegi_p['Price']/vegi_p['NDB Quantity']

vegi_p.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
vegi_prices = vegi_p.groupby('Food')['NDB Price'].min()

In [42]:
# solve for minimum cost diets for vegetarians

tol = 1e-2 # Numbers in solution smaller than this (in absolute value) treated as zeros

c = vegi_prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([-Amin,Amax])

b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

# Now solve problem!
result = lp(c, A, b, method='interior-point')

# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))
print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())

Cost of diet for F 19-30 is $3.48 per day.

You'll be eating (in 100s of grams or milliliters):
Food
Almond Milk                                              0.358236
Avocados                                                 4.395457
Bananas, per lb. (453.6 gm)                              1.710185
Almonds                                                  0.337327
Lentils                                                  1.108161
Cabbage                                                  0.481098
Nutritional Yeast                                        0.020455
Spinach                                                  1.438925
Beans, dried, any type, all sizes, per lb. (453.6 gm)    0.473858
Bread, whole wheat, pan, per lb. (453.6 gm)              0.808709
dtype: float64

With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000000       

In [43]:
# solve for vegans

# Convert food quantities to NDB units
veg_p['NDB Quantity'] = veg_p[['Quantity','Units']].T.apply(lambda x : ndb.ndb_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
veg_p['NDB Price'] = veg_p['Price']/veg_p['NDB Quantity']

veg_p.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
veg_prices = veg_p.groupby('Food')['NDB Price'].min()

In [44]:
# solve for minimum cost diets for vegans

tol = 1e-2 # Numbers in solution smaller than this (in absolute value) treated as zeros

c = veg_prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([-Amin,Amax])

b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

# Now solve problem!
result = lp(c, A, b, method='interior-point')

# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))
print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())

Cost of diet for F 19-30 is $3.48 per day.

You'll be eating (in 100s of grams or milliliters):
Food
Almond Milk                                              0.358236
Avocados                                                 4.395457
Bananas, per lb. (453.6 gm)                              1.710185
Almonds                                                  0.337327
Lentils                                                  1.108161
Cabbage                                                  0.481098
Nutritional Yeast                                        0.020455
Spinach                                                  1.438925
Beans, dried, any type, all sizes, per lb. (453.6 gm)    0.473858
Bread, whole wheat, pan, per lb. (453.6 gm)              0.808709
dtype: float64

With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2399.999999       

## Recipes

## 3 Dishes using ingredients for each type of minimum cost diet
### Omnivore: California Liver Roll or Pig Liver Congee
### Vegetarian/Vegan: Spicy Lentil Mash w/ Whole Wheat Bread

#### Both: Avocado Cabbage Salad
#### Snacks: Bananas, Almonds

### Notes: any additional ingredients needed (add to cost)
* Salt (4 cents per oz)
* Pepper (82 cents per oz)
* Curry powder (86 cents per oz)

## Total Cost for Population

In [28]:
# compute total cost and quantity to feed certain population with mix of age/sex groups (can use population project data)

In [29]:
# Get age/sex counts for the United States in 2017
# get list of all possible countries and years
age_range_codes = []
# Ranges are in 5 year sets up to 80
for i in range(0,80,5):
    age_range_codes.append(f"{i:02d}"+f"{i+4:02d}")
age_range_codes.append("80UP")

variable_labels ={"SP.POP.TOTL":"Population, total"}
total_pop_df = wbdata.get_dataframe(variable_labels) 
total_pop_df = total_pop_df.dropna().reset_index()[["country", "date"]]
total_pop_df["date"] = total_pop_df["date"].astype(int)
total_pop_df["country_code"] = total_pop_df["country"].apply(lambda row: wbdata.search_countries(row, display = False)[0]['id'])


def inner_pop(year, age_code, place, people):  
    if people == 'females':
        gender = 'FE'
    if people == 'males':
        gender = 'MA'
    
    data = 'SP.POP' + '.' + age_code + '.' + gender
    
    variable_labels = {data :"Population"}
        
    if place == 'world':
        place_code = 'WLD' # fixes bug where search also returns arab world
    else:
        place_code = wbdata.search_countries(place, display = False)[0]['id'] # finds country code for place listed
    
    try: 
        df = wbdata.get_dataframe(variable_labels, country=[place_code]) 
    except:
        print('Error: The place you listed may not be in the World Bank database or resulted in multiple possible countries or regions. Call wbdata.search_countries(place) to see the possible options and then refine your place query accordingly.')
    
    df.index = df.index.astype(int)
    
    population = df.loc[year, :][0]
    
    return int(population)


def population_df(regions = "all", years = "all"):
    
    pop_df = total_pop_df
    if type(regions) != list and regions != "all":
        regions = [regions]
    region_codes = []
    for region in regions:
        region_codes.append(wbdata.search_countries(region, display = False)[0]['id'])
    if type(years) != list and years != "all":
        years = [years]
    if regions == "all" and years == "all":
        for age in age_range_codes:
            pop_df[age + 'MA'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'males'), axis = 1)
            pop_df[age + 'FE'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'females'), axis = 1)
    elif regions == "all" and years != "all":
        pop_df = pop_df[pop_df['date'].isin(years)].reset_index(drop=True)
        for year in years:
            for age in age_range_codes:
                pop_df[age + 'MA'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'males'), axis = 1)
                pop_df[age + 'FE'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'females'), axis = 1)
    elif years == "all" and regions != "all":
        pop_df = pop_df[pop_df['country_code'].isin(region_codes)].reset_index(drop=True)
        for region in regions:
            for age in age_range_codes:
                pop_df[age + 'MA'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'males'), axis = 1)
                pop_df[age + 'FE'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'females'), axis = 1)
    else:
        pop_df = pop_df[(pop_df['date'].isin(years))&(pop_df['country_code'].isin(region_codes))].reset_index(drop=True)
        for region in regions:
            for year in years:
                for age in age_range_codes:
                    pop_df[age + 'MA'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'males'), axis = 1)
                    pop_df[age + 'FE'] = pop_df.apply(lambda row: inner_pop(row["date"], age, row["country"], 'females'), axis = 1)
    pop_df['country'] = pop_df['country'].str.lower()
    pop_df.set_index(['country', 'date'], inplace=True)
    pop_df.drop(columns = ['country_code'], inplace = True)
    return pop_df
usa = population_df(['united states'], [2017])
usa

,,0004MA,0004FE,0509MA,0509FE,1014MA,1014FE,1519MA,1519FE,2024MA,2024FE,...,6064MA,6064FE,6569MA,6569FE,7074MA,7074FE,7579MA,7579FE,80UPMA,80UPFE
country,date,,,,,,,,,,,,,,,,,,,,,
united states,2017,10123826,9676632,10624312,10158892,10740556,10279316,10853401,10423372,11661144,11177860,...,9601635,10238782,7939154,8810296,5728630,6741741,3868301,4753069,4825690,7537631


In [49]:
# assume uniform distribution of ages within age range to establish new age ranges to fit nutritional data
ages = {}
keys = ['C 1-3', 'F 4-8', 'M 4-8', 'F 9-13', 'M 9-13', 'F 14-18', 'M 14-18', 'F 19-30', 'M 19-30', 'F 31-50', 'M 31-50', 'F 51+', 'M 51+']

# 1-3
ages[keys[0]] = 3/4 * usa.iloc[0, 0] + 3/4 * usa.iloc[0, 1]
# 4-8
ages[keys[1]] = 1/4 * usa.iloc[0, 1] + 3/4* usa.iloc[0, 3]
ages[keys[2]] = 1/4 * usa.iloc[0, 0] + 3/4* usa.iloc[0, 2]
# 9-13
ages[keys[3]] = 1/4 * usa.iloc[0, 3] + 3/4* usa.iloc[0, 5]
ages[keys[4]] = 1/4 * usa.iloc[0, 2] + 3/4* usa.iloc[0, 4]
# 14-18
ages[keys[5]] = 1/4 * usa.iloc[0, 5] + 3/4* usa.iloc[0, 7]
ages[keys[6]] = 1/4 * usa.iloc[0, 4] + 3/4* usa.iloc[0, 6]
# 19-30
ages[keys[7]] = 1/4 * usa.iloc[0, 7] + usa.iloc[0, 9] + usa.iloc[0, 11] + 1/4 * usa.iloc[0, 13]
ages[keys[8]] = 1/4 * usa.iloc[0, 6] + usa.iloc[0, 8] + usa.iloc[0, 10] + 1/4 * usa.iloc[0, 12]
# 31-50
ages[keys[9]] = 1/4 * usa.iloc[0, 13] + usa.iloc[0, 15] + usa.iloc[0, 17] + usa.iloc[0, 19]
ages[keys[10]] = 1/4 * usa.iloc[0, 12] + usa.iloc[0, 14] + usa.iloc[0, 16] + usa.iloc[0, 18]
# 51+
ages[keys[11]] = 3/4 * usa.iloc[0, 21] + usa.iloc[0, 23] + usa.iloc[0, 25] + usa.iloc[0, 27] + usa.iloc[0, 29] + usa.iloc[0, 31] + usa.iloc[0, 33]
ages[keys[12]] = 3/4 * usa.iloc[0, 20] + usa.iloc[0, 22] + usa.iloc[0, 24] + usa.iloc[0, 26] + usa.iloc[0, 28] + usa.iloc[0, 30] + usa.iloc[0, 32]

ages

{'C 1-3': 14850343.5,
 'F 4-8': 10038327.0,
 'M 4-8': 10499190.5,
 'F 9-13': 10249210.0,
 'M 9-13': 10711495.0,
 'F 14-18': 10387358.0,
 'M 14-18': 10825189.75,
 'F 19-30': 27862129.5,
 'M 19-30': 28984147.25,
 'F 31-50': 33305728.5,
 'M 31-50': 33395588.0,
 'F 51+': 57263372.5,
 'M 51+': 50923636.25}

In [31]:
# loop through sex/age groups:
n = 0
costs = np.array([0] * 39)
costs = costs.astype(float)
for diet_prices in [omni_prices, vegi_prices, veg_prices]:
    for group in list(ages.keys()):
        # Define *minimums*
        bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

        # Define *maximums*
        bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]


        # solve for minimum cost diet

        tol = 1e-2 # Numbers in solution smaller than this (in absolute value) treated as zeros

        c = diet_prices.apply(lambda x:x.magnitude).dropna()

        # Compile list that we have both prices and nutritional info for; drop if either missing
        use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
        c = c[use]

        # Drop nutritional information for foods we don't know the price of,
        # and replace missing nutrients with zeros.
        Aall = D[c.index].fillna(0)

        # Drop rows of A that we don't have constraints for.
        Amin = Aall.loc[bmin.index]

        Amax = Aall.loc[bmax.index]

        # Minimum requirements involve multiplying constraint by -1 to make <=.
        A = pd.concat([-Amin,Amax])

        b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

        # Now solve problem!
        result = lp(c, A, b, method='interior-point')
        costs[n] = result.fun 
        n = n+1
        # Put back into nice series
        diet = pd.Series(result.x,index=c.index)
        
        print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

Cost of diet for C 1-3 is $1.31 per day.
Cost of diet for F 4-8 is $1.60 per day.
Cost of diet for M 4-8 is $1.50 per day.
Cost of diet for F 9-13 is $2.63 per day.
Cost of diet for M 9-13 is $2.52 per day.
Cost of diet for F 14-18 is $2.84 per day.
Cost of diet for M 14-18 is $2.75 per day.
Cost of diet for F 19-30 is $2.26 per day.
Cost of diet for M 19-30 is $2.32 per day.
Cost of diet for F 31-50 is $2.32 per day.
Cost of diet for M 31-50 is $2.35 per day.
Cost of diet for F 51+ is $2.92 per day.
Cost of diet for M 51+ is $2.35 per day.
Cost of diet for C 1-3 is $2.41 per day.
Cost of diet for F 4-8 is $2.47 per day.
Cost of diet for M 4-8 is $2.33 per day.
Cost of diet for F 9-13 is $4.08 per day.
Cost of diet for M 9-13 is $3.74 per day.
Cost of diet for F 14-18 is $4.27 per day.
Cost of diet for M 14-18 is $4.50 per day.
Cost of diet for F 19-30 is $3.48 per day.
Cost of diet for M 19-30 is $4.15 per day.
Cost of diet for F 31-50 is $3.62 per day.
Cost of diet for M 31-50 is $4.

In [32]:
df = pd.DataFrame({'Omnivores': costs[0:13], 'Vegetarians': costs[13:26], 'Vegans': costs[26:39]})
df.index = list(ages.keys())
df['USA Population in 2017'] = list(ages.values())
df

,Omnivores,Vegetarians,Vegans,USA Population in 2017
C 1-3,1.307326,2.409688,2.409688,14850343.50
F 4-8,1.603603,2.467128,2.467128,10038327.00
M 4-8,1.499916,2.328522,2.328521,10499190.50
F 9-13,2.629885,4.077082,4.246407,10249210.00
M 9-13,2.517160,3.740784,3.971882,10711495.00
F 14-18,2.840988,4.265007,4.382013,10387358.00
M 14-18,2.750629,4.496713,4.540923,10825189.75
F 19-30,2.260096,3.479948,3.479948,27862129.50
M 19-30,2.322775,4.147369,4.147343,28984147.25
F 31-50,2.320168,3.619675,3.619675,33305728.50


In [48]:
# most expensive diet: Male Vegans between 14-18
print('Cost per day:', df['Vegans']['M 14-18'])

Cost per day: 4.54092313600337


In [33]:
omni_cost = 0
for n in list(range(12)):
    omni_cost += df.iloc[n, 0] * df.iloc[n, 3]
print('Total Cost of Minimum Cost Omnivore Diet for U.S. Population in 2017:', '${:,.2f}'.format(omni_cost))
print('Average Cost of Minimum Cost Omnivore Diet for U.S. Population in 2017:', '${:,.2f}'.format(omni_cost / sum(df.iloc[:, 3])))

Total Cost of Minimum Cost Omnivore Diet for U.S. Population in 2017: $617,424,192.71
Average Cost of Minimum Cost Omnivore Diet for U.S. Population in 2017: $2.00


In [34]:
vegi_cost = 0
for n in list(range(12)):
    vegi_cost += df.iloc[n, 1] * df.iloc[n, 3]
print('Total Cost of Minimum Cost Vegetarian Diet for U.S. Population in 2017:', '${:,.2f}'.format(vegi_cost))
print('Average Cost of Minimum Cost Vegetarian Diet for U.S. Population in 2017:', '${:,.2f}'.format(vegi_cost / sum(df.iloc[:, 3])))

Total Cost of Minimum Cost Vegetarian Diet for U.S. Population in 2017: $984,577,121.23
Average Cost of Minimum Cost Vegetarian Diet for U.S. Population in 2017: $3.18


In [35]:
veg_cost = 0
for n in list(range(12)):
    veg_cost += df.iloc[n, 2] * df.iloc[n, 3]
print('Total Cost of Minimum Cost Vegan Diet for U.S. Population in 2017:', '${:,.2f}'.format(veg_cost))
print('Average Cost of Minimum Cost Vegan Diet for U.S. Population in 2017:', '${:,.2f}'.format(veg_cost / sum(df.iloc[:, 3])))

Total Cost of Minimum Cost Vegan Diet for U.S. Population in 2017: $990,481,541.51
Average Cost of Minimum Cost Vegan Diet for U.S. Population in 2017: $3.20


## Conclusion

### See presentation.